In [460]:
import json
import os
from pprint import pprint
import nltk
from nltk.tag.stanford import StanfordNERTagger
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import math
import operator
from pycorenlp import StanfordCoreNLP
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.tree import Tree
from collections import *
from gensim.models import word2vec
import gensim, logging
import tensorflow
from nltk.corpus import wordnet

 
nlp = StanfordCoreNLP('http://localhost:9000')

with open('training.json') as data_file:    
    train_data = json.load(data_file)
    
with open('development.json') as data_file:    
    dev_data = json.load(data_file)
    
with open('testing.json') as data_file:    
    test_data = json.load(data_file)

#pprint(data)
stop_words = set(stopwords.words('english'))


In [152]:
text = (
  'Pusheen and Smitha walked along the beach. '
  'Pusheen wanted to surf, but fell off the surfboard.')
output = nlp.annotate(text, properties={
  'annotators': 'tokenize,ssplit,pos,depparse,parse',
  'outputFormat': 'json'
  })
output['sentences'][0]['tokens'][0]['pos']



'NNP'

In [38]:
vect = TfidfVectorizer(min_df=1)
tfidf = vect.fit_transform(['Pusheen and Smitha walked along the beach. ',
                           'Pusheen wanted to surf, but fell off the surfboard.'])
(tfidf * tfidf.T).A

array([[ 1.        ,  0.14587439],
       [ 0.14587439,  1.        ]])

In [461]:
def getCorpus(data):
    contexts = []
    questions = []
    answers = []
    for i in range(0, len(data['data'])):
    #for i in range(0, 1):
        dataset = data['data'][i]
        for j in range(len(dataset['paragraphs'])):
        #for j in range(0, 1):
            question_bucket = dataset['paragraphs'][j]
            contexts.append(question_bucket['context'])
            for k in range(len(question_bucket['qas'])):
                questions.append(question_bucket['qas'][k]['question'])
                answers.append(question_bucket['qas'][k]['answers'][0]['text'])
    return contexts, questions, answers

train_contexts, train_questions, train_answers = getCorpus(train_data)
dev_contexts, dev_questions, dev_answers = getCorpus(dev_data)
test_contexts, test_questions, test_answers = getCorpus(test_data)

In [462]:
# import modules & set up logging
import gensim, logging
import string

translator = str.maketrans('', '', string.punctuation)

def getVectorizedCorpusModel(contexts, questions, answers):
    contexts_filtered = [c.split('.') for c in contexts]
    contexts_out = [d.translate(translator).split() for c in contexts_filtered for d in c]
    questions_out = [c.translate(translator).split() for c in questions]
    answers_out = [c.translate(translator).split() for c in answers]
    out = contexts_out + questions_out + answers_out
    return gensim.models.Word2Vec(out, size=200, window=5, min_count=1, workers=4)

model = getVectorizedCorpusModel(train_contexts + dev_contexts + test_contexts,
                                 train_questions + dev_questions + test_questions,
                                 train_answers + test_questions)

2017-11-04 19:31:35,441 : INFO : collecting all words and their counts
2017-11-04 19:31:35,447 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-11-04 19:31:35,494 : INFO : PROGRESS: at sentence #10000, processed 166897 words, keeping 21853 word types
2017-11-04 19:31:35,567 : INFO : PROGRESS: at sentence #20000, processed 336603 words, keeping 34412 word types
2017-11-04 19:31:35,647 : INFO : PROGRESS: at sentence #30000, processed 521823 words, keeping 44726 word types
2017-11-04 19:31:35,705 : INFO : PROGRESS: at sentence #40000, processed 704393 words, keeping 53541 word types
2017-11-04 19:31:35,766 : INFO : PROGRESS: at sentence #50000, processed 889949 words, keeping 62130 word types
2017-11-04 19:31:35,825 : INFO : PROGRESS: at sentence #60000, processed 1072611 words, keeping 69472 word types
2017-11-04 19:31:35,887 : INFO : PROGRESS: at sentence #70000, processed 1260605 words, keeping 76791 word types
2017-11-04 19:31:35,951 : INFO : PROGRESS: a

In [314]:
output = None
for t in train_contexts:
    output = nlp.annotate(t, properties={
      'annotators': 'pos,ner',
      'outputFormat': 'json'
      })

{'IN', 'VBN', 'RB', 'VB', '.', 'MD', 'PRP$', 'NNS', 'CC', ',', 'NNP', 'VBP', 'JJ', 'VBZ', 'TO', 'NN', 'DT', ':', 'PRP'}
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOCATION', 'O', 'LOCATION', 'O', 'LOCATION', 'LOCATION', 'O', 'LOCATION', 'O', 'LOCATION', 'O', 'LOCATION', 'O', 'LOCATION', 'LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PERSON', 'PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOCATION', 'O']


[0.021739130434782608,
 0.10869565217391304,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8695652173913043]

In [463]:
### filter out non-content words
def remove_stop_words(sentence):
    result = ''
    for t in sentence.split():
        if t.lower() not in stop_words:
            result += (t + ' ')
    return result.rstrip()

def calculateSimilarity(a, b):
    context_sentences = [a, b]
    tfidf = vect.fit_transform(context_sentences)
    prod = (tfidf * tfidf.T).A
    return prod[0][1]
    
def getPosNerList(sentence):
    output = nlp.annotate(sentence, properties={
      'annotators': 'pos,ner',
      'outputFormat': 'json'
      })
    posList = []
    nerList = []
    for s in output['sentences']:
        for t in s['tokens']:
            posList.append(t['pos'])
            nerList.append(t['ner'])
    return posList,nerList
            
def getNerFracVector(annotateList):
    nerType = {'PERSON': 0, 'LOCATION': 1, 'ORGANIZATION': 2, 'DATE': 3, 'TIME': 4, 'MONEY': 5, 'PERCENT': 6, 'MISC': 7, 'O': 8}
    freq = [0] * len(nerType)
    for a in annotateList:
        if a in nerType:
            freq[nerType[a]] += 1
    
    if len(annotateList) > 0:
        for i in range(len(freq)):
            freq[i] /= (1.0 * len(annotateList))
    return freq
    
def getPosFracVector(annotateList):
    posType = {'NNP': 0, 'NNPS': 1, 'NN': 2, 'NNS': 3}
    freq = [0] * len(posType)
    for a in annotateList:
        if a in posType:
            freq[posType[a]] += 1
    
    if len(annotateList) > 0:
        for i in range(len(freq)):
            freq[i] /= (1.0 * len(annotateList))
    return freq
    
def getQuestionTypeVector(question):
    qType = {'what': 0, 'where': 1, 'which': 2, 'who': 3, 'when': 4, 'why': 5, 'how': 6}
    qWord = 'what'
    for d in question:
        if d.lower() in qType:
            qWord = d.lower()
    vec = [0] * 7
    vec[qType[qWord]] = 1
    return vec

def getFeatureForQuestionAndContext(question, q_pos, q_ner, context, c_pos, c_ner):
    q = question.translate(translator).split()
    c = context.translate(translator).split()
    
    feat = sum(model[k] for k in q) / len(q)
    if len(c) > 0:
        feat += sum(model[k] for k in c) / len(c)
    
    # Use word embeddings for current question
    #feat = (sum(model[k] for k in q) / len(q) + sum(model[k] for k in c) / len(c)).tolist()
    feat = feat.tolist()
    
    # Add similarity bewteen question and current sentence
    feat.append(calculateSimilarity(question, context))

    # Add question Type
    feat += getQuestionTypeVector(q)

    # Add pos and ner overlap between question and context sentence
    #q_pos, q_ner = getPosNerList(remove_stop_words(question))
    #c_pos, c_ner = getPosNerList(remove_stop_words(context))

    q_pos_frac = getPosFracVector(q_pos)
    c_pos_frac = getPosFracVector(c_pos)
    posOverlap = [0] * len(q_pos_frac)
    for i in range(len(posOverlap)):
        posOverlap[i] = q_pos_frac[i] * c_pos_frac[i]

    feat += posOverlap

    q_ner_frac = getNerFracVector(q_ner)
    c_ner_frac = getNerFracVector(c_ner)
    nerOverlap = [0] * len(q_ner_frac)
    for i in range(len(nerOverlap)):
        nerOverlap[i] = q_ner_frac[i] * c_ner_frac[i]

    feat += nerOverlap

    '''print(question)
    print(q_pos)
    print(q_pos_frac)
    print(q_ner)
    print(q_ner_frac)


    print(context)
    print(c_pos)
    print(c_pos_frac)
    print(c_ner)
    print(c_ner_frac)

    print(posOverlap)
    print(nerOverlap)

    print(len(q))
    print(feat)'''
    #return np.array(feat)
    return feat

#contexts = [d.strip() for d in data['data'][0]['paragraphs'][0]['context'].split('.')]
#question = data['data'][0]['paragraphs'][0]['qas'][0]['question']
#X = getFeatureForQuestionAndContext(question, contexts[0])

In [379]:
y = [0] * len(contexts)
ans = data['data'][0]['paragraphs'][0]['qas'][4]['answers'][0]['text']
for i in range(len(y)):
    if ans in contexts[i]:
        y[i] = 1
y

[0, 1, 0, 0, 0]

In [517]:
#contexts = [d.strip() for d in data['data'][0]['paragraphs'][0]['context'].split('.')]
#question = data['data'][0]['paragraphs'][0]['qas'][0]['question']
#X = getFeatureForQuestionAndContext(question, contexts[0])

def getFeatureAndLabelForParagraph(paragraph):
    contexts = [d.strip() for d in paragraph['context'].split('.') if len(d) > 0]
    X = []
    y = []
    c_pos = []
    c_ner = []
    for i in range(len(contexts)):
        c_pos_i, c_ner_i = getPosNerList(remove_stop_words(contexts[i]))
        c_pos.append(c_pos_i)
        c_ner.append(c_ner_i)
        
    for qas in paragraph['qas']:
        question = qas['question']
        answer = qas['answers'][0]['text']
        q_pos, q_ner = getPosNerList(remove_stop_words(question))

        for i in range(len(contexts)):
            if len(contexts[i]) > 0 and len(question) > 0:
                X.append(getFeatureForQuestionAndContext(question, q_pos, q_ner, contexts[i], c_pos[i], c_ner[i]))
                y.append(1 if answer in contexts[i] else 0)
    return X, y, len(contexts)

def getFeatureAndLabel(data):
    X = []
    y = []
    counter = 0

    for i in range(0, len(data['data'])):
        if counter > 500:
            break
    #for i in range(0, 1):
        dataset = data['data'][i]
        for j in range(len(dataset['paragraphs'])):
        #for j in range(0, 1):
            counter += 1
            if counter % 500 == 0:
                print(counter)
            question_bucket = dataset['paragraphs'][j]
            Xj, yj, _ = getFeatureAndLabelForParagraph(question_bucket)
            X += Xj
            y += yj
    return X, y
#X0, y0 = getFeatureAndLabelForParagraph(data['data'][0]['paragraphs'][0])
#X1, y1 = getFeatureAndLabelForParagraph(data['data'][0]['paragraphs'][1])

In [518]:
X, y = getFeatureAndLabel(train_data)

500


In [519]:
def getFeatureForParagraph(paragraph):
    contexts = [d.strip() for d in paragraph['context'].split('.') if len(d) > 0]
    X = []
    c_pos = []
    c_ner = []
    for i in range(len(contexts)):
        c_pos_i, c_ner_i = getPosNerList(remove_stop_words(contexts[i]))
        c_pos.append(c_pos_i)
        c_ner.append(c_ner_i)
        
    for qas in paragraph['qas']:
        question = qas['question']
        q_pos, q_ner = getPosNerList(remove_stop_words(question))

        for i in range(len(contexts)):
            if len(contexts[i]) > 0 and len(question) > 0:
                X.append(getFeatureForQuestionAndContext(question, q_pos, q_ner, contexts[i], c_pos[i], c_ner[i]))
    return X, len(contexts)

def getFeature(data):
    X = []
    counter = 0

    for i in range(0, len(data['data'])):
    #for i in range(0, 1):
        dataset = data['data'][i]
        for j in range(len(dataset['paragraphs'])):
        #for j in range(0, 1):
            counter += 1
            if counter % 100 == 0:
                print(counter)
            question_bucket = dataset['paragraphs'][j]
            Xj, context_size = getFeatureForParagraph(question_bucket)
            X += Xj
    return X

In [520]:
len(y)

9596

In [521]:
import numpy as np
from sklearn import linear_model
X = np.array(X)
y = np.array(y)

#clf = linear_model.SGDClassifier()
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=1)
clf.fit(X, y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [522]:
testX, testY, context_size = getFeatureAndLabelForParagraph(dev_data['data'][0]['paragraphs'][2])
print(clf.predict(testX))
result = clf.predict(testX)
print(np.array(testY))
print(context_size)

[1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0]
[0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 1 1
 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0
 1]
3


In [523]:
result = np.resize(result, (int(len(result) / context_size), context_size))
result.shape

(25, 3)

In [524]:
result.shape[0]

25

In [525]:
import random

def generate_answer_for_file(file_name, model):
    with open(file_name) as data_file:    
        data = json.load(data_file)
    our_pred = {}
    for i in range(0, len(data['data'])):
    #for i in range(0, 1):
        dataset = data['data'][i]
        for j in range(len(dataset['paragraphs'])):
        #for j in range(0, 1):
            question_bucket = dataset['paragraphs'][j]
            X_prime, context_size = getFeatureForParagraph(question_bucket)
            y_pred = model.predict(X_prime)
            y_pred = np.resize(y_pred, (int(len(y_pred) / context_size), context_size))
            contexts = [d.strip() for d in question_bucket['context'].split('.') if len(d) > 0]

            for k in range(len(question_bucket['qas'])):
                question = question_bucket['qas'][k]['question']
                qid = question_bucket['qas'][k]['id']
                #our_pred[qid] = answer
                candIndex = -1
                if k >= y_pred.shape[0]:
                    candIndex = random.choice(range(context_size))
                else:
                    for l in range(len(y_pred[k])):
                        if y_pred[k][l] == 1:
                            candIndex = l
                            break;
                    if candIndex == -1:
                        candIndex = random.choice(range(context_size))
                    
                our_pred[qid] = contexts[candIndex]

    return our_pred

In [526]:
our_pred = generate_answer_for_file('development.json', clf)

In [527]:
with open('pred_development.json', 'w') as outfile:
    json.dump(our_pred, outfile)

In [358]:
data['data'][0]['paragraphs'][0]

{'context': 'A pub /pʌb/, or public house is, despite its name, a private house, but is called a public house because it is licensed to sell alcohol to the general public. It is a drinking establishment in Britain, Ireland, New Zealand, Australia, Canada, Denmark and New England. In many places, especially in villages, a pub can be the focal point of the community. The writings of Samuel Pepys describe the pub as the heart of England.',
 'qas': [{'answers': [{'answer_start': 105,
     'text': 'it is licensed to sell alcohol'}],
   'id': '56dede3c3277331400b4d784',
   'question': 'What is a pub licensed to sell?'},
  {'answers': [{'answer_start': 393, 'text': 'the pub'}],
   'id': '56dede3c3277331400b4d785',
   'question': 'In many villages what establishment could be called the focal point of the community?'},
  {'answers': [{'answer_start': 16, 'text': 'public house'}],
   'id': '56dfb4987aa994140058e003',
   'question': "What is the term 'pub' short for?"},
  {'answers': [{'answer_st

In [210]:
model['pub']

array([-0.00145477,  0.00253437,  0.00426356,  0.0032987 , -0.00381742,
       -0.00113026,  0.00283679,  0.00299052, -0.0005114 ,  0.0009465 ,
        0.00310909,  0.00136164,  0.00442152,  0.00102174, -0.00294037,
        0.00392738, -0.0022461 ,  0.00126364,  0.00200181, -0.00328614,
       -0.00325674,  0.00141178, -0.00187451,  0.00371888, -0.0019388 ,
        0.00343885,  0.00489268,  0.00273538,  0.00060415, -0.00419724,
        0.00175545, -0.0012204 ,  0.0010134 , -0.00355457, -0.00191119,
       -0.00049069, -0.00327332,  0.00034621, -0.00088512,  0.000737  ,
       -0.00431812,  0.00413071, -0.00044786,  0.00357864,  0.00268949,
       -0.00320058, -0.0028346 , -0.00068395,  0.0026268 , -0.00312221,
        0.00366472,  0.00118429,  0.00095956,  0.00397674, -0.00308599,
        0.00235966, -0.00461775,  0.00357801,  0.00182779, -0.00073156,
        0.00489754, -0.00193946, -0.00405051, -0.00223823,  0.00201041,
       -0.00232397,  0.00427489, -0.00122443, -0.00175184, -0.00

In [196]:
context = data['data'][0]['paragraphs'][0]['context']
question = data['data'][0]['paragraphs'][0]['qas'][3]['question']
context_sentences = [c.lstrip() for c in context.split('.') if len(c) > 0]
context_sentences.append(question)
print(context_sentences)
tfidf = vect.fit_transform(context_sentences)
prod = (tfidf * tfidf.T).A
size = prod.shape[0]
print(prod)
condidate_sentence_index = np.argmax(prod[size - 1][:size - 1])
print(context_sentences[condidate_sentence_index])

output = nlp.annotate(question, properties={
  'annotators': 'pos,ner,parse',
  'outputFormat': 'json'
  })
print(output['sentences'][0]['parse'])
parsestr = output['sentences'][0]['parse']
for i in Tree.fromstring(parsestr).subtrees():
    if i.label() == 'NP':
        print(' '.join(i.leaves()))

output = nlp.annotate(context_sentences[condidate_sentence_index], properties={
  'annotators': 'pos,ner,parse',
  'outputFormat': 'json'
  })
print(output['sentences'][0]['parse'])
parsestr = output['sentences'][0]['parse']
for i in Tree.fromstring(parsestr).subtrees():
    if i.label() == 'NP':
        print(' '.join(i.leaves()))

['A pub /pʌb/, or public house is, despite its name, a private house, but is called a public house because it is licensed to sell alcohol to the general public', 'It is a drinking establishment in Britain, Ireland, New Zealand, Australia, Canada, Denmark and New England', 'In many places, especially in villages, a pub can be the focal point of the community', 'The writings of Samuel Pepys describe the pub as the heart of England', 'Where in the United States are pubs located?']
[[ 1.          0.10203864  0.04592331  0.06078796  0.01876728]
 [ 0.10203864  1.          0.06300119  0.04680124  0.0439374 ]
 [ 0.04592331  0.06300119  1.          0.28424234  0.16231839]
 [ 0.06078796  0.04680124  0.28424234  1.          0.10329405]
 [ 0.01876728  0.0439374   0.16231839  0.10329405  1.        ]]
In many places, especially in villages, a pub can be the focal point of the community
(ROOT
  (SBARQ
    (WHADVP (WRB Where)
      (PP (IN in)
        (NP (DT the) (NNP United) (NNPS States))))
    (SQ

In [181]:
# import modules & set up logging
print(word2idx)
print(idx2word)


{'': 0, 'licensed': 1, 'and': 2, 'new': 3, 'places,': 4, 'private': 5, 'focal': 6, 'but': 7, 'drinking': 8, 'heart': 9, 'england': 10, 'sell': 11, 'is,': 12, 'called': 13, 'despite': 14, 'in': 15, 'many': 16, 'describe': 17, 'name,': 18, 'point': 19, 'britain,': 20, 'canada,': 21, 'australia,': 22, 'especially': 23, 'or': 24, 'because': 25, 'pub': 26, 'to': 27, 'writings': 28, 'its': 29, 'community': 30, 'a': 31, 'public': 32, 'house,': 33, 'ireland,': 34, 'general': 35, 'it': 36, 'alcohol': 37, 'establishment': 38, 'denmark': 39, 'pepys': 40, 'samuel': 41, 'villages,': 42, 'can': 43, 'zealand,': 44, '/pʌb/,': 45, 'of': 46, 'as': 47, 'be': 48, 'house': 49, 'is': 50, 'the': 51}
['', 'licensed', 'and', 'new', 'places,', 'private', 'focal', 'but', 'drinking', 'heart', 'england', 'sell', 'is,', 'called', 'despite', 'in', 'many', 'describe', 'name,', 'point', 'britain,', 'canada,', 'australia,', 'especially', 'or', 'because', 'pub', 'to', 'writings', 'its', 'community', 'a', 'public', 'hous

In [259]:
model.most_similar(positive=['Denmark'], negative=[], topn=3)
#sum(model['England'])
model.similarity('Denmark', 'England')

0.087885368218185836

In [90]:
parsestr = output['sentences'][0]['parse']
for i in Tree.fromstring(parsestr).subtrees():
    if i.label() == 'NP':
        print(i)

(NP (JJ many) (NNS places))
(NP (NNS villages))
(NP (DT a) (NN pub))
(NP
  (NP (DT the) (JJ focal) (NN point))
  (PP (IN of) (NP (DT the) (NN community))))
(NP (DT the) (JJ focal) (NN point))
(NP (DT the) (NN community))


In [329]:
#text = ('Alice lives in England in January 2017')
text = ('While in Cornell University in January 2017, at 1:30 AM, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.')
output = nlp.annotate(text, properties={
  'annotators': 'pos,ner,parse',
  'outputFormat': 'json'
  })

In [330]:
output['sentences'][0]['tokens']

[{'after': ' ',
  'before': '',
  'characterOffsetBegin': 0,
  'characterOffsetEnd': 5,
  'index': 1,
  'lemma': 'while',
  'ner': 'O',
  'originalText': 'While',
  'pos': 'IN',
  'word': 'While'},
 {'after': ' ',
  'before': ' ',
  'characterOffsetBegin': 6,
  'characterOffsetEnd': 8,
  'index': 2,
  'lemma': 'in',
  'ner': 'O',
  'originalText': 'in',
  'pos': 'IN',
  'word': 'in'},
 {'after': ' ',
  'before': ' ',
  'characterOffsetBegin': 9,
  'characterOffsetEnd': 16,
  'index': 3,
  'lemma': 'Cornell',
  'ner': 'ORGANIZATION',
  'originalText': 'Cornell',
  'pos': 'NNP',
  'word': 'Cornell'},
 {'after': ' ',
  'before': ' ',
  'characterOffsetBegin': 17,
  'characterOffsetEnd': 27,
  'index': 4,
  'lemma': 'University',
  'ner': 'ORGANIZATION',
  'originalText': 'University',
  'pos': 'NNP',
  'word': 'University'},
 {'after': ' ',
  'before': ' ',
  'characterOffsetBegin': 28,
  'characterOffsetEnd': 30,
  'index': 5,
  'lemma': 'in',
  'ner': 'O',
  'originalText': 'in',
  'pos

In [35]:
method_list = [func for func in dir(StanfordCoreNLP) if callable(getattr(StanfordCoreNLP, func))]
method_list

['__class__',
 '__delattr__',
 '__dir__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'annotate',
 'regex',
 'semgrex',
 'tokensregex']

In [15]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('/Users/shuohuang/Downloads/stanford-tools/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz',
					   '/Users/shuohuang/Downloads/stanford-tools/stanford-ner/stanford-ner.jar',
					   encoding='utf-8')

text = 'While in Cornell University in January 2017, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)


print(classified_text)

[('While', 'O'), ('in', 'O'), ('Cornell', 'ORGANIZATION'), ('University', 'ORGANIZATION'), ('in', 'O'), ('January', 'O'), ('2017', 'O'), (',', 'O'), ('Christine', 'PERSON'), ('Lagarde', 'PERSON'), ('discussed', 'O'), ('short-term', 'O'), ('stimulus', 'O'), ('efforts', 'O'), ('in', 'O'), ('a', 'O'), ('recent', 'O'), ('interview', 'O'), ('with', 'O'), ('the', 'O'), ('Wall', 'ORGANIZATION'), ('Street', 'ORGANIZATION'), ('Journal', 'ORGANIZATION'), ('.', 'O')]


In [275]:
q_type = {'what':'NN', 'who':'NNP'}

### filter out non-content words
def remove_stop_words(sentence):
    result = ''
    for t in sentence.split():
        if t.lower() not in stop_words:
            result += (t + ' ')
    return result.rstrip()

### get all the content words in a sentence
def get_content_token_set(sentence):
    token_set = set()
    for pt in sentence.translate(translator).split():
        if pt.lower() not in stop_words:
            token_set.add(pt.lower())
    return token_set

### check the number of overlaps from context sentence tokens with question sentence tokens
def compare(q_text_set, c_text_set):
    score = 0
    for c_token in c_text_set:
        if c_token in q_text_set:
            score += 1
    return score

### find the sentence from context corpus which has the most overlaps with the question
def find_candidate_sentence(q_token_set, context):
    candidate = context[0]
    score = compare(q_token_set, get_content_token_set(candidate))
    for i in range(1, len(context)):
        temp_candi = context[i]
        temp_score = compare(q_token_set, get_content_token_set(temp_candi))
        if temp_score >= score:
            score = temp_score
            candidate = temp_candi
    return candidate

### define question type
def find_question_type(question):
    for q in question.split():
        if q.lower() == 'what' or q.lower() == 'which':
            return 'NN'
        elif q.lower() == 'who' or q.lower() == 'where':
            return 'NNP'

### 
def find_possible_answer_with_distance(q_token_set, candidate_sentence, q_type):
    token_positions = {}
    cand_tokens = candidate_sentence.split()
    for i in range(len(cand_tokens)):
        if cand_tokens[i].lower() in q_token_set:
            token_positions[cand_tokens[i]] = i
    #print(token_positions)
    c_tag = nltk.pos_tag(word_tokenize(candidate_sentence))
    possible_answers = set()
    for c in c_tag:
        if c[1] == q_type and c[0].lower() not in q_token_set:
            possible_answers.add(c[0])
    
    answer_possitions = {}
    for i in range(len(cand_tokens)):
        if cand_tokens[i] in possible_answers:
            answer_possitions[cand_tokens[i]] = i
    answer_score = {}
    for ap in answer_possitions:
        position = answer_possitions[ap]
        score = 0
        for tp in token_positions:
            score += abs(position - token_positions[tp])
        answer_score[ap] = score
    answer_score = sorted(answer_score.items(), key=operator.itemgetter(1))
    #print(possible_answers)
    #print(answer_possitions)
    #print(answer_score)
    
    answer = str()
    for ans in answer_score:
        answer += (ans[0] + ' ')
    
    return answer.rstrip()

def find_answer(question, context):
    q_tag = nltk.pos_tag(word_tokenize(question))
    q_token_set = get_content_token_set(question)
    q_type = find_question_type(question)
    context_sentences = [c.lstrip().translate(translator) for c in context.split('.') if len(c) > 0]
    candidate_sentence = find_candidate_sentence(q_token_set, context_sentences)
    #c_tag = nltk.pos_tag(word_tokenize(candidate_sentence))
    answer = find_possible_answer_with_distance(q_token_set, candidate_sentence, q_type)
    #print(question)
    #print(q_type)
    #print(q_token_set)
    #print(candidate_sentence)
    #print(c_tag)
    #print(answer)
    return answer

In [276]:
def generate_answer_for_file(file_name):
    with open(file_name) as data_file:    
        data = json.load(data_file)
    our_pred = {}
    for i in range(0, len(data['data'])):
    #for i in range(0, 1):
        dataset = data['data'][i]
        for j in range(len(dataset['paragraphs'])):
        #for j in range(0, 1):
            question_bucket = dataset['paragraphs'][j]
            context = question_bucket['context']
            for k in range(len(question_bucket['qas'])):
                question = question_bucket['qas'][k]['question']
                qid = question_bucket['qas'][k]['id']
                answer = find_answer(question, context)
                our_pred[qid] = answer
    return our_pred

In [277]:
our_pred = generate_answer_for_file('development.json')
with open('pred_development.json', 'w') as outfile:
    json.dump(our_pred, outfile)
    

our_pred = generate_answer_for_file('testing.json')
with open('pred_testing.json', 'w') as outfile:
    json.dump(our_pred, outfile)